In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# 讀取CSV文件
df = pd.read_csv('/content/drive/MyDrive/archive/dogs.csv')

# 選擇指定類別
selected_breeds = [
    "Airedale", "Beagle", "Bloodhound", "Bluetick", "Chihuahua",
    "Collie", "Dingo", "French Bulldog", "German Sheperd",
    "Malinois", "Newfoundland", "Pekinese", "Pomeranian", "Pug", "Vizsla"
]

# 過濾指定的品種
df_selected = df[df['labels'].isin(selected_breeds)]

# 加載和預處理圖像
def load_images(df):
    image_list = []
    for idx, row in df.iterrows():
        img = Image.open('/content/drive/MyDrive/archive/' + row['filepaths'])
        img = img.resize((224, 224))  # 假設使用224x224大小的圖像
        img_array = np.array(img) / 255.0  # 圖像歸一化
        image_list.append(img_array)
    return np.array(image_list)

images = load_images(df_selected)

# 標籤編碼
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_selected['labels'])
labels = to_categorical(labels)  # 轉換為one-hot編碼

# 分割數據集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# 建立CNN模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(selected_breeds), activation='softmax')
])

# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練模型，使用数据增强
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=10,
          validation_data=(X_test, y_test))

# 評估模型
val_loss, val_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {val_acc}")


Epoch 1/10
51/51 [==============================] - 244s 5s/step - loss: 2.9146 - accuracy: 0.1068 - val_loss: 2.5753 - val_accuracy: 0.1340
Epoch 2/10
51/51 [==============================] - 238s 5s/step - loss: 2.5028 - accuracy: 0.1664 - val_loss: 2.3471 - val_accuracy: 0.2060
Epoch 3/10
51/51 [==============================] - 242s 5s/step - loss: 2.3919 - accuracy: 0.2011 - val_loss: 2.2405 - val_accuracy: 0.2333
Epoch 4/10
51/51 [==============================] - 237s 5s/step - loss: 2.2944 - accuracy: 0.2148 - val_loss: 2.0982 - val_accuracy: 0.3077
Epoch 5/10
51/51 [==============================] - 238s 5s/step - loss: 2.2914 - accuracy: 0.2402 - val_loss: 2.0928 - val_accuracy: 0.3127
Epoch 6/10
51/51 [==============================] - 237s 5s/step - loss: 2.2279 - accuracy: 0.2502 - val_loss: 1.9916 - val_accuracy: 0.3151
Epoch 7/10
51/51 [==============================] - 231s 4s/step - loss: 2.1902 - accuracy: 0.2737 - val_loss: 1.9923 - val_accuracy: 0.3548
Epoch 8/10
51